
Dask Dataframes on NYC Taxi Data
================================

<img src="http://pandas.pydata.org/_static/pandas_logo.png"
     align="left"
     width="30%"
     alt="Pandas logo">
     <img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo">

In this section we will learn how to ...

-  use Dask Dataframe to scale Pandas workloads
-  call `.compute` and `.persist` to trigger computation
-  start and scale a Dask cluster on Kubernetes
-  interpret dashboard plots


## We have several CSV files in cloud storage

In [ ]:
from gcsfs import GCSFileSystem
gcs = GCSFileSystem()

sorted(gcs.glob('anaconda-public-data/nyc-taxi/csv/2015/yellow_*.csv'))

## Read a subset with Pandas

It's too big to fit in memory on a single machine, so we pull out the first million rows to get a first impression.

In [ ]:
import pandas as pd

with gcs.open('anaconda-public-data/nyc-taxi/csv/2015/yellow_tripdata_2015-01.csv') as f:
    df = pd.read_csv(f, nrows=1000000, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [ ]:
df.head()

## Investigate the subset as normal

In [ ]:
len(df)  # How many rides in the dataset

In [ ]:
df.passenger_count.sum()  # How many passengers total?

In [ ]:
df.passenger_count.value_counts()  # What was the distribution of passengers?

In [ ]:
df.groupby(df.passenger_count).trip_distance.mean()  # What was the average trip distance, grouped by passenger

In [ ]:
# Tip Fraction, grouped by hour-of-day
df2 = df[(df.tip_amount > 0) & (df.fare_amount > 0)]
df2['tip_fraction'] = df2.tip_amount / df2.fare_amount
hour = df2.groupby(df2.tpep_pickup_datetime.dt.hour).tip_fraction.mean()

In [ ]:
%matplotlib inline

hour.plot(figsize=(10, 6), title='Tip Fraction by Hour')

## Start a Dask Cluster

Your notebook is conveniently attached to a Kubernetes cluster, so you can start a Dask cluster using the [dask-kubernetes](https://kubernetes.dask.org/en/latest/) project.

For more information on deploying Dask on different cluster technology see [Dask's deployment documentation](https://docs.dask.org/en/latest/setup.html)

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=20)
cluster

In [ ]:
from dask.distributed import Client

client = Client()

## Create Dask dataframe around all of the data

Before we loaded only a subset of one CSV file.  Now lets use Dask dataframe to read all of the files.

For more information you can read [Dask's documentation for creating dataframes](http://docs.dask.org/en/latest/dataframe-create.html)

In [ ]:
import dask.dataframe as dd

df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2015/yellow_*.csv', 
                 parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
df

Dask dataframes look like Pandas dataframes, and support most of the common Pandas methods.

In [ ]:
df.passenger_count.sum()

## Investigate laziness

Note that this did not yet load the data into memory.  Dask dataframes are *lazy* by default, so they only evaluate when we tell them to.

There are two ways to trigger computation:

-  `result = result.compute()`: triggers computation and stores the result into local memory as a Pandas object.  You should use this with *small* results that will fit into memory.
-  `result = result.persist()`: triggers computation and stores the result into distributed memory, returning another Dask dataframe object.  You should use this with *large* results that you want to stage in distributed memory for repeated computation.

#### *Task*: determine how many passengers there were in 2015

#### *Task*: determine the distribution of passengers

How many rides were there with one passenger, two passengers, and so on?

How does this compare to our sample from before?

#### *Task*: determine the average trip distance, grouped by passenger count

How does this compare to our sample?

#### *Task*: What did our workers spend their time doing?

To answer this question look at the Task Stream dashboard plot.  It will tell you the activity on each core of your cluster (y-axis) over time (x-axis).  You can hover over each rectangle of this plot to determine what kind of task it was.  What kinds of tasks are most common and take up the most time?

*TODO*: Add image of task stream plot

*Extra*: if you're ahead of the group you might also want to look at the Profile dashboard plot.  You can access this by selecting the orange Dask icon on the left side of your JupyterLab page.

## Persist data in memory

Each time we call compute we re-evaluate the entire computation, starting with downloading data from cloud storage, reading that CSV data into Pandas dataframes, and then doing the normal Pandas groupbys and such that we care about.

It is common to do some initial data preparation work (loading and parsing data) and then want to persist that data into distributed memory for faster access.  You can do this with the `.persist` method.

    result = result.persist()
    
Do this now and then repeat your computations from before.  How does this affect computation time?

#### *Task*: Persist your dataframe into memory, then repeat the earlier computations



In [ ]:
# Persist df


In [ ]:
# Determine the distribution of passengers


In [ ]:
# Determine how many passengers there were in 2015


In [ ]:
# Determine the average trip distance, grouped by passenger count
